In [37]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=ECVYXC3Qam62ToRzRWym5vLqUGabOh&access_type=offline&code_challenge=DLFiZWD7ZFNOfXpWxNNp4qXTLnUcabHe6jDGeSphzco&code_challenge_method=S256


Credentials saved to file: [/Users/yt4/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "open-targets-genetics-dev" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some Google Clo

In [2]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=8M4N53U9yCWP5e3xhGRJiRhz0GYZoi&access_type=offline&code_challenge=9NaZYMw0TTpILf0buJ3Rxx9JSPyw1GK-WChqbAIRjLI&code_challenge_method=S256


You are now logged in as [yt4@sanger.ac.uk].
Your current project is [open-targets-genetics-dev].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [1]:
import os

import hail as hl
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql import DataFrame

from gentropy.common.session import Session
from gentropy.dataset.study_index import StudyIndex
from gentropy.dataset.summary_statistics import SummaryStatistics
from gentropy.dataset.study_locus import StudyLocus
from gentropy.susie_finemapper import SusieFineMapperStep
from gentropy.method.drug_enrichment_from_evid import chemblDrugEnrichment

Loading BokehJS ...

In [2]:
"""Common utilities for the project."""

import os
from pathlib import Path
from gentropy.common.session import Session
import logging


def get_gcs_credentials() -> str:
    """Get the credentials for google cloud storage."""
    app_default_credentials = os.path.join(
        os.getenv("HOME", "."), ".config/gcloud/application_default_credentials.json"
    )

    service_account_credentials = os.path.join(
        os.getenv("HOME", "."), ".config/gcloud/service_account_credentials.json"
    )

    if Path(app_default_credentials).exists():
        return app_default_credentials
    else:
        raise FileNotFoundError("No GCS credentials found.")


def get_gcs_hadoop_connector_jar() -> str:
    """Get the google cloud storage hadoop connector for spark.

    This function will return the url to download the hadoop jar.
    """

    return (
        "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar"
    )


def gcs_conf(
    credentials_path=None, project="open-targets-genetics-dev"
) -> dict[str, str]:
    """Get the spark configuration with hadoop connector for google cloud storage."""
    credentials_path = credentials_path or get_gcs_credentials()
    return {
        "spark.driver.memory": "12g",
        "spark.kryoserializer.buffer.max": "500m",
        "spark.driver.maxResultSize":"2g",
        "spark.hadoop.fs.gs.impl": "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem",
        "spark.jars": get_gcs_hadoop_connector_jar(),
        "spark.hadoop.google.cloud.auth.service.account.enable": "true",
        "spark.hadoop.fs.gs.project.id": project,
        "spark.hadoop.google.cloud.auth.service.account.json.keyfile": credentials_path,
        "spark.hadoop.fs.gs.requester.pays.mode": "AUTO",
    }


class GentropySession(Session):
    def __init__(self, *args, **kwargs):
        if "extended_spark_conf" in kwargs:
            kwargs["extended_spark_conf"].update(gcs_conf())
        else:
            kwargs["extended_spark_conf"] = gcs_conf()
        super().__init__(*args, **kwargs)

    @property
    def conf(self):
        logging.warning(
            "To change the config restart the session and use the `extended_spark_conf` parameter."
        )
        return self.spark.sparkContext.getConf().getAll()

session= GentropySession()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/25 13:58:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/25 13:58:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/25 13:58:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/25 13:58:34 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
path_to_release_folder="gs://open-targets-data-releases/25.06/"

si=StudyIndex.from_parquet(session,path_to_release_folder+"output/study/")
sl=StudyLocus.from_parquet(session,path_to_release_folder+"output/credible_set/")

# LITERATURE - SKIP

In [4]:
evidence=session.spark.read.parquet("gs://open-targets-data-releases/25.06/output/evidence")

In [5]:
dd= evidence.groupBy("datasourceId").count()
dd.show()

+------------------+--------+
|      datasourceId|   count|
+------------------+--------+
|         europepmc|21641315|
|               eva| 3138383|
|              impc| 1175688|
|gwas_credible_sets| 1082677|
|            chembl|  573124|
|  expression_atlas|  229404|
|  genomics_england|   34784|
|       gene_burden|   36805|
|       eva_somatic|    9719|
|cancer_gene_census|   82754|
|        slapenrich|   72406|
|          reactome|   10162|
|  uniprot_variants|   33047|
|     crispr_screen|   21711|
|           intogen|    4224|
|          orphanet|    6293|
|uniprot_literature|    6714|
|    gene2phenotype|    4181|
| cancer_biomarkers|    1300|
|           clingen|    3016|
+------------------+--------+
only showing top 20 rows



In [6]:
evidence.show(1)

25/05/08 18:04:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+---------------+-------------+-------------------+--------+----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+----------+--------------------+--------+-------------+---------------------+--------------+-----------------+--------+----------------+---------------+----------+--------+-------------------+----------+----------------+-----------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+------+------------+-----------------+----------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+--------------+--------------+--------+---------+----------+------------+-----------+--------------+-------------+----+------------------------+-----------------+-------------------------+----

In [5]:
evidence = evidence.filter(f.col("score")>=0.95).filter(f.col("datasourceId")=="europepmc").cache()
evidence.count()

25/06/24 18:56:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


110373

In [6]:
filtered_pairs = evidence.groupBy("diseaseId", "targetId") \
    .agg(f.count("*").alias("pair_count")) \
    .filter(f.col("pair_count") >= 2) \
    .select("diseaseId", "targetId")

# Show the result
filtered_pairs.count()

14510

In [9]:
filtered_pairs.distinct().count()

14143

In [7]:
sgl_l=filtered_pairs.cache()
sgl_l.count()

14510

In [11]:
#sgl_l=evidence.select("diseaseId","targetId").distinct().cache()
#sgl_l.count()

In [8]:
sgl_l.show(2)

+-------------+---------------+
|    diseaseId|       targetId|
+-------------+---------------+
|MONDO_0004975|ENSG00000148120|
|  EFO_0003060|ENSG00000265681|
+-------------+---------------+
only showing top 2 rows



# OTHER RARE VARINAT EVIDENCE

In [4]:
evidence=session.spark.read.parquet(path_to_release_folder+"/output/evidence")

In [5]:
evidence = evidence.filter(f.col("score")>=0.95).filter(f.col("datasourceId").isin(["eva", "uniprot_variants", "gene2phenotype", "genomics_england", "clingen","uniprot_literature","orphanet"])).cache()
evidence.count()


25/06/25 13:58:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


138400

In [6]:
dd= evidence.groupBy("datasourceId").count()
dd.show()

+------------------+-----+
|      datasourceId|count|
+------------------+-----+
|               eva|57746|
|  genomics_england|31037|
|  uniprot_variants|32368|
|          orphanet| 6289|
|uniprot_literature| 6239|
|           clingen| 1968|
|    gene2phenotype| 2753|
+------------------+-----+



In [7]:
evidence.show(1)

+------------+---------------+-------------------+-------------------+--------+----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+----------+--------------------+--------+-------------+---------------------+--------------+-----------------+--------+--------------------+---------------+--------------------+--------+-------------------+-------------------+----------------+--------------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+------+------------+-----------------+--------------------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+--------------+--------------+--------+---------+----------+------------+-----------+--------------+-------------+----+------------------------+------

In [8]:
sgl0=evidence.select("diseaseId","targetId").distinct().cache()
sgl0.count()

16371

## CHEMBL

In [9]:
chembl=session.spark.read.parquet(path_to_release_folder+"/output/evidence/sourceId=chembl")

In [10]:
chembl.show(1)

+------------+---------------+-------------+-------------------+--------+----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+----------+--------------------+--------+-------------+---------------------+--------------+-----------------+--------+----------------+---------------+----------+--------+-------------------+----------+----------------+--------------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+----------+------------+-----------------+----------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+--------------+--------------+--------+---------+----------+------------+-----------+--------------+-------------+----+------------------------+-----------------+-----------------------

In [11]:
chembl.printSchema()

root
 |-- datasourceId: string (nullable = true)
 |-- targetId: string (nullable = true)
 |-- alleleOrigins: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- allelicRequirements: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ancestry: string (nullable = true)
 |-- ancestryId: string (nullable = true)
 |-- beta: double (nullable = true)
 |-- betaConfidenceIntervalLower: double (nullable = true)
 |-- betaConfidenceIntervalUpper: double (nullable = true)
 |-- biologicalModelAllelicComposition: string (nullable = true)
 |-- biologicalModelGeneticBackground: string (nullable = true)
 |-- biologicalModelId: string (nullable = true)
 |-- biomarkerName: string (nullable = true)
 |-- biomarkers: struct (nullable = true)
 |    |-- geneExpression: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |-- genetic

In [12]:
clinical_phase_counts = chembl.groupBy("clinicalPhase").count()
clinical_phase_counts.show()

+-------------+------+
|clinicalPhase| count|
+-------------+------+
|          1.0|126478|
|          4.0|104823|
|          0.5|  7183|
|          3.0|122304|
|          2.0|212315|
+-------------+------+



In [13]:
chembl4=chembl.filter((f.col("clinicalPhase") == 4) | (f.col("clinicalPhase") == 3)) #&(f.col("clinicalStatus")=="Completed")
chembl4=chembl4.select("diseaseId","targetId","clinicalStatus","clinicalPhase","drugId").cache()
chembl4.count()

227127

In [14]:
#clinical_phase_counts = chembl4.filter(f.col("clinicalPhase")==4).groupBy("clinicalStatus").count()
#clinical_phase_counts.show()

In [15]:
sgl=chembl4.select("diseaseId","targetId").distinct().cache()
sgl.count()

25685

In [17]:
#sgl=session.spark.read.parquet("gs://genetics-portal-dev-analysis/xg1/Chembl_l2g_goldstandards/EGL_GSP_2409.parquet")
#sgl=session.spark.read.parquet("gs://genetics-portal-dev-analysis/xg1/Effector_gene_list.parquet")
#sgl=session.spark.read.parquet("gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/EGL_old_l2g_indirect_0.7")

#sgl=sgl.withColumnRenamed("efo_terms","diseaseId")
#sgl=sgl.withColumnRenamed("geneId","targetId")
#sgl.count()

# Old GS

In [18]:
sgl2=session.spark.read.json("gs://genetics-portal-dev-analysis/yt4/2506_release/otg_gs_230511.json")

In [19]:
sgl2.printSchema()

root
 |-- association_info: struct (nullable = true)
 |    |-- otg_id: string (nullable = true)
 |-- gold_standard_info: struct (nullable = true)
 |    |-- gene_id: string (nullable = true)
 |    |-- highest_confidence: string (nullable = true)
 |-- metadata: struct (nullable = true)
 |    |-- set_label: string (nullable = true)
 |-- sentinel_variant: struct (nullable = true)
 |    |-- alleles: struct (nullable = true)
 |    |    |-- alternative: string (nullable = true)
 |    |    |-- reference: string (nullable = true)
 |    |-- locus_GRCh38: struct (nullable = true)
 |    |    |-- chromosome: string (nullable = true)
 |    |    |-- position: long (nullable = true)
 |-- trait_info: struct (nullable = true)
 |    |-- ontology: array (nullable = true)
 |    |    |-- element: string (containsNull = true)



In [20]:
sgl2.count()

1279

In [21]:
sgl2.show(1,truncate=False)

+----------------+-----------------------+-------------+-----------------------+---------------+
|association_info|gold_standard_info     |metadata     |sentinel_variant       |trait_info     |
+----------------+-----------------------+-------------+-----------------------+---------------+
|{GCST006061_3}  |{ENSG00000168356, High}|{ot_platform}|{{A, C}, {3, 38592651}}|{[EFO_0000275]}|
+----------------+-----------------------+-------------+-----------------------+---------------+
only showing top 1 row



In [22]:
sgl2.show(1,truncate=False)

+----------------+-----------------------+-------------+-----------------------+---------------+
|association_info|gold_standard_info     |metadata     |sentinel_variant       |trait_info     |
+----------------+-----------------------+-------------+-----------------------+---------------+
|{GCST006061_3}  |{ENSG00000168356, High}|{ot_platform}|{{A, C}, {3, 38592651}}|{[EFO_0000275]}|
+----------------+-----------------------+-------------+-----------------------+---------------+
only showing top 1 row



In [23]:
sgl2.groupBy("gold_standard_info.highest_confidence").count().show()

+------------------+-----+
|highest_confidence|count|
+------------------+-----+
|              High|  629|
|            Medium|  650|
+------------------+-----+



In [24]:
selected_df = sgl2.filter(f.col("gold_standard_info.highest_confidence").isin(["High","Medium"])).select( 
    f.col("gold_standard_info.gene_id").alias("geneId"),
    f.col("trait_info.ontology").alias("ontology")
)

# Show the results
selected_df.show(1,truncate=False)

+---------------+-------------+
|geneId         |ontology     |
+---------------+-------------+
|ENSG00000168356|[EFO_0000275]|
+---------------+-------------+
only showing top 1 row



In [25]:
exploded_df = selected_df.withColumn("efo_terms", f.explode("ontology")).drop("ontology")

exploded_df=exploded_df.withColumnRenamed("efo_terms","diseaseId").withColumnRenamed("geneId","targetId").select("diseaseId","targetId").distinct()
# Show the results
exploded_df.show(1,truncate=False)

+-------------+---------------+
|diseaseId    |targetId       |
+-------------+---------------+
|MONDO_0005090|ENSG00000140009|
+-------------+---------------+
only showing top 1 row



In [26]:
exploded_df.count()

812

# Combining

In [27]:
sgl.count()

25685

In [28]:
sgl.show(1)

+-----------+---------------+
|  diseaseId|       targetId|
+-----------+---------------+
|EFO_0001645|ENSG00000101981|
+-----------+---------------+
only showing top 1 row



In [29]:
exploded_df.count()

812

In [30]:
exploded_df.show(1)

+-------------+---------------+
|    diseaseId|       targetId|
+-------------+---------------+
|MONDO_0005090|ENSG00000140009|
+-------------+---------------+
only showing top 1 row



In [31]:
sgl0.count()

16371

In [32]:
sgl0.show(1)

+-------------+---------------+
|    diseaseId|       targetId|
+-------------+---------------+
|MONDO_0008170|ENSG00000185345|
+-------------+---------------+
only showing top 1 row



In [33]:
columns_order = exploded_df.columns

sgl_reordered = sgl.select(columns_order)
sgl0_reordered = sgl0.select(columns_order)
#shl_l_reordered = sgl_l.select(columns_order)


In [34]:
combined_df = exploded_df.union(sgl_reordered).union(sgl0_reordered).distinct().cache()
#combined_df = exploded_df.union(sgl_reordered).union(sgl0_reordered).union(shl_l_reordered).distinct().cache()
combined_df.show(1,truncate=False)

+-------------+---------------+
|diseaseId    |targetId       |
+-------------+---------------+
|MONDO_0005090|ENSG00000140009|
+-------------+---------------+
only showing top 1 row



In [35]:
combined_df.count()

42288

In [36]:
combined_df=combined_df.dropDuplicates(["targetId","diseaseId"]).cache()
combined_df.count()

42288

In [37]:
combined_df.write.mode("overwrite").parquet("gs://genetics-portal-dev-analysis/yt4/2506_release/training_set/20250625_EGL_2506_0.95_otg_chembl.parquet")

# Combine with target - SKIP

In [46]:
egl=session.spark.read.parquet("gs://genetics-portal-dev-analysis/yt4/2506_release/training_set/EGL_all.parquet")

In [47]:
egl.count()

65002

In [48]:
egl.show(1,truncate=False)

+-----------+---------------+
|diseaseId  |targetId       |
+-----------+---------------+
|EFO_0004530|ENSG00000173334|
+-----------+---------------+
only showing top 1 row



In [49]:
target=session.spark.read.parquet(path_to_release_folder+"output/target/")

In [50]:
target=target.withColumnRenamed("id","targetId")

In [51]:
egl = egl.join(target, on="targetId", how="left").cache()
egl.count()

65002

In [52]:
egl.show(1)

+---------------+-----------+--------------+--------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+---------------+-------------+--------------------+----+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------+
|       targetId|  diseaseId|approvedSymbol|       biotype|       transcriptIds| canonicalTranscript|      canonicalExons|     genomicLocation|alternativeGenes|        approvedName|                  go|hallmarks|            synonyms|      symbolSynonyms|        nameSynonyms|functionDescriptions|subcellularLocations|targetClass|obsoleteSymbols|obsoleteNames|          constraint| tep|          proteinIds|             dbXrefs|chemicalProbes|          homologues|    

In [53]:
egl.select("targetId","biotype").distinct().count()

8334

In [54]:
egl.groupBy("biotype").count().show()

+--------------------+-----+
|             biotype|count|
+--------------------+-----+
|      protein_coding|64176|
|               miRNA|   54|
|              lncRNA|  408|
|           IG_V_gene|   46|
|transcribed_proce...|    6|
|           IG_C_gene|   30|
|             Mt_tRNA|   74|
|processed_pseudogene|   56|
|     IG_V_pseudogene|    2|
|              snoRNA|    4|
|transcribed_unpro...|   14|
|           TR_C_gene|    6|
|transcribed_unita...|   47|
|unprocessed_pseud...|   30|
|                    |    4|
|               scRNA|    2|
|           TR_V_gene|   18|
|               snRNA|   13|
|            ribozyme|    5|
|             Mt_rRNA|    6|
+--------------------+-----+
only showing top 20 rows



In [55]:
64176/65002

0.9872926986861943

In [56]:
egl1=egl.select("targetId","diseaseId","biotype","genomicLocation.chromosome","genomicLocation.start").distinct().cache()
egl1.count()

65002

In [57]:
egl.select("diseaseId").distinct().count()

9636

In [58]:
egl1.show()

+---------------+-------------+--------------+----------+---------+
|       targetId|    diseaseId|       biotype|chromosome|    start|
+---------------+-------------+--------------+----------+---------+
|ENSG00000070182|  EFO_0005558|protein_coding|        14| 64746283|
|ENSG00000070182|MONDO_0017319|protein_coding|        14| 64746283|
|ENSG00000070182|MONDO_0019350|protein_coding|        14| 64746283|
|ENSG00000070182|MONDO_0009948|protein_coding|        14| 64746283|
|ENSG00000070182|MONDO_0011411|protein_coding|        14| 64746283|
|ENSG00000070182|MONDO_0002280|protein_coding|        14| 64746283|
|ENSG00000070182|   HP_0004839|protein_coding|        14| 64746283|
|ENSG00000070182|MONDO_0012511|protein_coding|        14| 64746283|
|ENSG00000072071|MONDO_0100038|protein_coding|        19| 14147743|
|ENSG00000072071|   HP_0001249|protein_coding|        19| 14147743|
|ENSG00000072071|  EFO_0003888|protein_coding|        19| 14147743|
|ENSG00000072071|   HP_0001328|protein_coding|  

In [59]:
egl1=egl1.toPandas()

In [60]:
egl1

,targetId,diseaseId,biotype,chromosome,start
0,ENSG00000070182,EFO_0005558,protein_coding,14,64746283
1,ENSG00000070182,MONDO_0017319,protein_coding,14,64746283
2,ENSG00000070182,MONDO_0019350,protein_coding,14,64746283
3,ENSG00000070182,MONDO_0009948,protein_coding,14,64746283
4,ENSG00000070182,MONDO_0011411,protein_coding,14,64746283
...,...,...,...,...,...
64997,ENSG00000196262,MONDO_0020290,protein_coding,7,44796680
64998,ENSG00000205581,EFO_0000616,protein_coding,21,39342315
64999,ENSG00000210164,HP_0001699,Mt_tRNA,MT,9991
65000,ENSG00000210164,HP_0003546,Mt_tRNA,MT,9991


In [61]:
egl1["close_to_other_gene"] = 0

In [63]:
egl1[egl1["chromosome"] == '1'].sort_values(by="start")

,targetId,diseaseId,biotype,chromosome,start,close_to_other_gene
56719,ENSG00000229344,MONDO_0015459,unprocessed_pseudogene,1,632757,0
56711,ENSG00000229344,EFO_0005537,unprocessed_pseudogene,1,632757,0
56712,ENSG00000229344,MONDO_0008903,unprocessed_pseudogene,1,632757,0
56721,ENSG00000229344,MONDO_0001056,unprocessed_pseudogene,1,632757,0
56720,ENSG00000229344,MONDO_0004992,unprocessed_pseudogene,1,632757,0
...,...,...,...,...,...,...
35569,ENSG00000162711,MONDO_0019751,protein_coding,1,247332331,0
35571,ENSG00000162711,EFO_0005854,protein_coding,1,247332331,0
35561,ENSG00000162711,EFO_0000199,protein_coding,1,247332331,0
8627,ENSG00000162722,EFO_1001951,protein_coding,1,247857187,0


In [64]:
egl1["close_to_other_gene"] = 0
delta=500000
for row in egl1[["diseaseId", "chromosome"]].drop_duplicates().itertuples(index=False):
    diseaseId, chrom = row.diseaseId, row.chromosome
    ind=(egl1["chromosome"] == chrom) & (egl1["diseaseId"]==diseaseId)
    if sum(ind)>1:
        chrom_df = egl1[ind].sort_values(by="start")
        for i in range(len(chrom_df)-1):
            if (abs(chrom_df.iloc[i]["start"] - chrom_df.iloc[i+1]["start"]) <= delta):
                egl1.loc[chrom_df.index[i], "close_to_other_gene"] = 1
                egl1.loc[chrom_df.index[i+1], "close_to_other_gene"] = 1


In [65]:
sum(egl1["close_to_other_gene"]==1)

14593

In [66]:
egl2=egl1[egl1["close_to_other_gene"]==0].copy()

In [67]:
egl2

,targetId,diseaseId,biotype,chromosome,start,close_to_other_gene
0,ENSG00000070182,EFO_0005558,protein_coding,14,64746283,0
1,ENSG00000070182,MONDO_0017319,protein_coding,14,64746283,0
2,ENSG00000070182,MONDO_0019350,protein_coding,14,64746283,0
3,ENSG00000070182,MONDO_0009948,protein_coding,14,64746283,0
4,ENSG00000070182,MONDO_0011411,protein_coding,14,64746283,0
...,...,...,...,...,...,...
64995,ENSG00000196262,EFO_1000388,protein_coding,7,44796680,0
64996,ENSG00000196262,EFO_0004254,protein_coding,7,44796680,0
64997,ENSG00000196262,MONDO_0020290,protein_coding,7,44796680,0
64998,ENSG00000205581,EFO_0000616,protein_coding,21,39342315,0


In [68]:
egl2_spark = session.spark.createDataFrame(egl2)

In [69]:
egl2_spark.count()

50409

In [70]:
egl2_spark.write.mode("overwrite").parquet("gs://genetics-portal-dev-analysis/yt4/2506_release/training_set/EGL_no_close_genes.parquet")

25/05/08 20:46:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1346932 ms exceeds timeout 120000 ms
25/05/08 20:46:42 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/08 20:46:42 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$